In [2]:
import nltk
import math
import os
import sys
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')

# получаем список стоп слов
stop = stopwords.words('english')
stop_ru = stopwords.words('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
inverted_index_folder = "/4  курс/Информационный поиск/task3"
tokens_folder = "/4  курс/Информационный поиск/task2"
lemmas_folder = "/4  курс/Информационный поиск/task4"
vectors_folder = "/4  курс/Информационный поиск/task5/vectors"

In [6]:
# соберем все леммы 
lemmas = []
f = open('/content/drive/My Drive/'+ tokens_folder +'/lemmas.txt', encoding='utf-8')
data = f.read()
lines = data.splitlines()
for line in lines:
  lemmas.append(line.split(':')[0])

print(lemmas)

['какой', 'российский', 'бумага', 'избавляться', 'крупный', 'акционер', 'год', 'промой', 'дарить', 'акция', 'регистрация', 'мочь', 'получать', 'учебник', 'сравнятор', 'сообщество', 'ипотека', 'кредитный', 'история', 'тест', 'дневник', 'трата', 'налоговый', 'вычет', 'написать', 'инвестиция', 'бизнес', 'разбор', 'закон', 'калькулятор', 'путешествие', 'медицина', 'здоровье', 'недвижимость', 'подкаста', 'видео', 'смотреть', 'сервис', 'расчет', 'закладка', 'письмо', 'оставаться', 'деньги', 'поток', 'инвестразбор', 'headhunter', 'group', 'plc', 'hhr', 'купить', 'адр', 'циан', 'cian', 'р', 'qiwi', 'спб', 'биржа', 'spbe', 'эл', 'энерго', 'enru', 'партнерство', 'тинькофф', 'котировка', 'обновляться', 'минута', 'фондовый', 'рынок', 'проходить', 'несколько', 'сделка', 'рассказывать', 'подробно', 'каждый', 'никита', 'компанищенко', 'частный', 'инвестор', 'профиль', 'автор', 'начало', 'январь', 'объявлять', 'смениться', 'контролировать', 'бывший', 'основной', 'собственник', 'инвестиционный', 'фонд'

In [15]:
# создадим файл с векторами (вот тут не понятно токены использовать или леммы)
for dirpath, _, filenames in os.walk('/content/drive/My Drive/'+ lemmas_folder):
  for filename in filenames:
    if "lemmas" not in filename:
      continue;
    dictionary = {}
    vector = {}
    file = open('/content/drive/My Drive/'+ lemmas_folder +'/' + filename, encoding='utf-8')
    data = file.read()
    lines = data.splitlines()

    for line in lines:
      word = line.split(' ')[0]
      idf = line.split(' ')[1]
      dictionary[word] = idf


    for lemma in lemmas:
      if lemma in dictionary.keys():
        vector[lemma] = dictionary[lemma]
      else: 
        vector[lemma] = 0

    with open('/content/drive/My Drive/' + vectors_folder + '/' + filename, 'w') as f:
      for key, idf_val in vector.items():
        f.write(f"{key} ")
        f.write(f"{idf_val} ")  
        f.write(f"\n")  

100


In [16]:
# конвертируем вектор обратно в словарь
def convert_file_to_vector(filename):
    vector_dict = {}
    file = open('/content/drive/My Drive/'+ vectors_folder +'/' + filename, encoding='utf-8')
    vecotor_strings = file.read().splitlines()
    file.close()
    for vecotor_string in vecotor_strings:
        if vecotor_string != "":
            vector_dict[vecotor_string.split(" ")[0]] = float(vecotor_string.split(" ")[1])
    return vector_dict

In [59]:
def vector_length(vector):
    return math.sqrt(sum(x ** 2 for x in vector.values()))

def compare_vectors(v1, v2):
    len = 0
    len_v1 = vector_length(v1)
    len_v2 = vector_length(v2)
    for lemma in lemmas:
        len = len + v1[lemma] * v2[lemma]
  
    if (len == 0 or len_v1 == 0 or len_v2 == 0):
      return 0

    return len / (math.sqrt(len_v1) * math.sqrt(len_v2))

In [60]:
def search(search_term):
    current_lemmas = set()
    search_dictionary = {}
    vector_dict = {}

    # file = open("../tokens")
    # tokens = file.read().split("\n")
    # file.close()

    for lemma in lemmas:
        search_dictionary[lemma] = 0

    # убираем стоп слова из поисковой строки
    search_term = '  '.join([word for word in search_term.split() if word not in (stop_ru)])
    words = search_term.split()

    # считаем вектор для поисковой строки 
    for word in words:
        current_lemma = WordNetLemmatizer().lemmatize(word)
        current_lemmas.add(current_lemma)
        search_dictionary[word] = search_dictionary[word] + 1

    for lemma in lemmas:
        vector_dict[lemma] = 0
        
    for lemma in current_lemmas:
        vector_dict[lemma] = search_dictionary[lemma] / float(len(words))

    result_dict = {}
    for dirpath, _, filenames in os.walk('/content/drive/My Drive/'+ vectors_folder):
      for filename in filenames:
        res = compare_vectors(convert_file_to_vector(filename), vector_dict);
        result_dict[filename] = res
    return result_dict

In [71]:
result = search('какой российский')
print(result)
print(sorted(result.items(), key=lambda item: item[1], reverse=True)) 

{'lemmas-32.txt': 0, 'lemmas-15.txt': 0, 'lemmas-14.txt': 0, 'lemmas-28.txt': 0.41943503557610073, 'lemmas-19.txt': 0.580416508681743, 'lemmas-7.txt': 0.21686934338603892, 'lemmas-29.txt': 0, 'lemmas-30.txt': 0.14221582271994537, 'lemmas-8.txt': 0, 'lemmas-9.txt': 0.2086563023963621, 'lemmas-24.txt': 0.16433403779778286, 'lemmas-25.txt': 0.26404047614246057, 'lemmas-4.txt': 0.04910549730752249, 'lemmas-13.txt': 0.13036029153724774, 'lemmas-39.txt': 0.21137177203387061, 'lemmas-47.txt': 0, 'lemmas-45.txt': 0.12090894354128107, 'lemmas-53.txt': 0, 'lemmas-16.txt': 0, 'lemmas-12.txt': 0.752499127289335, 'lemmas-52.txt': 0.3101241537681609, 'lemmas-3.txt': 0, 'lemmas-35.txt': 0, 'lemmas-36.txt': 0, 'lemmas-22.txt': 0, 'lemmas-51.txt': 0, 'lemmas-68.txt': 0, 'lemmas-92.txt': 0, 'lemmas-90.txt': 0.18165359074668128, 'lemmas-93.txt': 0.9059887390878729, 'lemmas-79.txt': 0.28620468467070664, 'lemmas-86.txt': 0.1354589352000665, 'lemmas-0.txt': 1.3576314546907462, 'lemmas-77.txt': 0.21085314239